# More on Flask
___dynamic endpoints, apis, and a sidenote on maps___

## Objective

We're going to experiment with a few other designs with flask, particularly:

* how we can use it for a blog or to host notebook html
* creating api endpoints to return calculated results from sklearn
* a quick introduction to maps with a small (and kind of wrong) implementation in flask

_(We say "wrong" because traditionally we should be doing map manipulation in javascript, but for the sake of brevity, we'll use python)_

## Class Notes

In the last lecture we were looking at how form logic works in flask in order to POST data. In particular we were looking at the `request` object:

```python
request.form['Lyrics']
```

Here the form is stored under the request object as an immutable dictionary (we don't want the request data to change!), so we can pull the data out and use it. The prediction was made, data inserted into the database, and redirected back to the main url.

This form allowed 'Lyrics' to be dynamic: to change according to the form POST. There are other ways to create dynamic endpoints: passing variables into the url, and passing a querystring.

### Variable endpoints

```python
@app.route('/blog/<page>')
def blog_post(page):
    md = open(page)
    try:
        return render_template('blog/markdown.html', text=md.read())
    except:
        return "Page does not exist!"
```

Above flask interprets <> from the route as a variable to be passed into the route method. We could make a very simple endpoint that, at the very least, takes whatever content in the endpoint and passes it as the webpage:

```python
@app.route('content/<content>')
def try_anything(content):
    return content
```

Objectively, this means two things:

1. We can create some method of finding all of our blog posts in some folder to create an index page
2. The index page can easily link to all of the blog posts which can use this variable endpoint.

#### Sidenote on saving and hosting notebooks:

There are two routes, in our current context, to host notebooks on your own. The first is using the built in html export tool, which uses the `mistune` python library and [pandoc](http://johnmacfarlane.net/pandoc/). You can do this from File -> Download as -> HTML, or using nbconvert at the command line:

```sh
ipython nbconvert --to html notebook.ipynb
```

The second route would be considerably more lightweight and more easily allow for your own css, using the markdown converter. This is not built into the notebook, but can be done with:

```sh
ipython nbconvert --to markdown notebook.ipynb
```

### Using querystrings

What if we wanted an endpoint that is considerably more dynamic, or in particular, used for an api response?
Our form example was perfect before, since we had a website in use, but in many contexts, we may not even need a web "page," but instead just a valid response to work with.

Our request object has another attribute called "args." The response to that could look like this:

```python
$ !curl localhost:5000/api/predict?text=yellow%20submarine
>>> ImmutableMultiDict([('text', u'yellow submarine')])
```

We'll use this alongside returning valid JSON to act as an "api," endpoints the flask server could use, or some other server could use. So instead of returning a whole web page, we could use this endpoint to get data instead:

```python
$ !curl localhost:5000/api/predict?text=yellow%20submarine
>>> '{"prediction": "The Beatles", "text": "yellow submarine"}' 
```

### Another machine learning tie in

Let's use the beer dataset from an earlier class (normalized) to compute some distances and build an api to "recommend" two other beers, using distances. Refer to the flask app for the task at hand.

## Maps

Maps are incredibly deep content, but let's surface understanding some basics in implementing maps and choropleths (interpolation of data). 

In [1]:
def inline_map(map):
    """
    Embeds the HTML source of the map directly into the IPython notebook.
    
    This method will not work if the map depends on any files (json data). Also this uses
    the HTML5 srcdoc attribute, which may not be supported in all browsers.
    """
    map._build_map()
    return HTML('<iframe srcdoc="{srcdoc}" style="width: 100%; height: 510px; border: none"></iframe>'.format(srcdoc=map.HTML.replace('"', '&quot;')))

In [2]:
import folium
from IPython.display import HTML 
map = folium.Map([40.7127, -74.0059])
inline_map(map)

### Why do we map?

Some common usecases for projects:

* visualizing location of data: student performance by district, airbnb locations, traffic
* understanding clusters of people and distances: Where are all of the C grade restaurants vs A grade? 
* Interpolating data points: averages and standard deviations from norms based a county, city, country
* Understanding customers: Where do they live? What markets haven't been reached?

### How do we map?

While the earth is technically a complex shape (closest shape would be an ellipsoid), we'll use projections of it as a flat space: this would look most like the dimensions of a scatter plot, and why many of us may have already attempted to visualize map data using latitude and longitude as x and y.

While intuitively this works, we'll be more interested in either:

1. Plotting markers or shapes: This would be for specific data points to plot
2. Generating choropleths, which shade and interpolate data onto shapes voer our map. This would be for generalizations.

### Plotting markers

Point maps are a better alternative for absolute values - the only geometry that they preserve is a single point for each feature.

The specific point or marker used in this style varies tremendously - Coloring points based on their sequential or categorical value can be useful, but points can also be scaled to different sizes to show their relative value. These scaled symbols can be any shape or image, such as circles, squares, or pictures of what they represent. In cases where there are multiple values that total up, scaled pie charts can be a terrific way to visualize what would otherwise be a complex dataset.

Care must be taken to not show too many points at once, as this will make a map difficult to read. In cases where there are too many points, a choropleth with aggregated values from the points may be a good alternative. Another alternative is to use clustering, where crowded points are grouped together until the map is zoomed in closer.

#### On geocoding

Explaining a point on a map isn't as simple as "oh, this latitiude and longitude." The geocoding process instead can get incredibly detailed, dependent on the data point we are interested in. For example, consider this geocoded response for New York Presbyterian:

http://www.datasciencetoolkit.org/maps/api/geocode/json?sensor=false&address=630+west+168th+street+new+york

In this case, we get a rehashing of the address level data, as well as some square approximation for the location of the building. 

Let's try using these long and lats as markers to our map:

In [3]:
import requests

hosp = requests.get('http://www.datasciencetoolkit.org/maps/api/geocode/json?sensor=false&address=630+west+168th+street+new+york')
hosp = hosp.json()
hosp = hosp['results'][0]['geometry']['location']
location = [hosp['lat'], hosp['lng']]

map = folium.Map(location, zoom_start=13, tiles="Stamen Toner")
map.circle_marker(location=location,radius=200,fill_opacity=0.5,fill_color='#3186cc')
inline_map(map)

Now, try the following:

1. use the datsciencetoolkit link to geocode a few other places (maybe your university, work, personal home) and plot them as markers. You can use and try circle_marker, polygon_marker, and simple_marker.
2. Look into the data csv `geoplaces2.csv`, which includes lat, lng, and some relevant categorical features.
    1. Determine where the data primarily lies, or filter down to a subset in a particular area.
    2. using markers, color code categories such as alcohol, smoking, or dress code, and place on the map using lat and long.
    3. Reconsider the best position to start the map based on the data you are using.

### Plotting choropleths

#### Sequential & Categorical

Symbolization tends to highlight two different characteristics of data: sequential and categorical. Sequential, or continuous data could also be called linear - it tends to be number values within a set range, like graduation rates between 0 and 100, or elevation. Categorical, or discrete data is, instead, one of a set number of values - like ‘true’, ‘false’, ‘democrat’ or ‘republican’.

This division between data is one of the main concerns for symbolization - a sequential datasource would fit with a scaled point map or a gradient color ramp on a raster, whereas categorical data generally is displayed using multiple symbols for markers, or discrete bucketing of colors.

![where heatmaps go wrong](http://imgs.xkcd.com/comics/heatmap.png)

[Color Brewer](http://colorbrewer2.org/), designed by Cynthina Brewer, is a great tool for devising colorizations that make sense dependent on if you are working with sequential or categorial data.


### Example

In [4]:
import pandas as pd
import folium 
loc = r'../../data'
notebook_loc = '../tree/data'
state_geo = notebook_loc + '/us-states.json'
state_unemployment = loc + r'/US_Unemployment_Oct2012.csv'

state_data = pd.read_csv(state_unemployment)

#Let Folium determine the scale
map = folium.Map(location=[48, -102], zoom_start=3)
map.geo_json(geo_path=state_geo, data=state_data,
             columns=['State', 'Unemployment'],
             threshold_scale=[5, 6, 7, 8, 9, 10],
             key_on='feature.id',
             fill_color='BuPu', fill_opacity=0.7, line_opacity=0.5,
             legend_name='Unemployment Rate (%)',
             reset=True)
inline_map(map)

### Try:

1. Changing the granularity based on the csv file. It's currently set to "State."
2. Changing the metric based on the other columns in the csv file.
3. Adjust the threshold scale.

### More Reading | Next Steps

* On the maps side, a more detailed explaination can be found at [Map School](http://www.macwright.org/mapschool/). Some of the above notes were borrowed from this source.
* Dive more into the [folium](https://github.com/python-visualization/folium) documentation to explore how you might use maps in your projects.
* [So you'd like to make a map using Python](http://sensitivecities.com/so-youd-like-to-make-a-map-using-python-EN.html#.VSV4qBPF8a5)
* introduction into [vincent](http://wrobstory.github.io/2013/04/python-maps-choropleth.html), another plotting tool available in python
